# 🎰 JetX AGGRESSIVE Model - Para Kaybı Önleyici

**⚠️ 1.5x ALTI TAHMİNLERİNDE YÜKSEK DOĞRULUK İÇİN OPTİMİZE EDİLDİ**

Bu notebook Google Colab'da çalışır ve agresif eğitim ile 1.5 altı tahminlerinde maksimum performans sağlar.

## Özellikler
- ✅ Class Imbalance çözümü (1.5 altı için 2.5x ağırlık)
- ✅ Focal Loss (yanlış tahminlere ekstra ceza)
- ✅ Güçlendirilmiş model (2x katman)
- ✅ 300 epoch agresif eğitim
- ✅ Detaylı metrik tracking
- ✅ Colab optimizasyonları

## Hedefler
- 1.5 ALTI Doğruluk: **70%+** ✅
- 1.5 ÜSTÜ Doğruluk: 75%+
- Genel Accuracy: 80%+

**Süre:** ~2-3 saat (GPU ile)

In [ ]:
# Kütüphaneleri yükle
!pip install -q tensorflow scikit-learn pandas numpy scipy joblib matplotlib seaborn tqdm

import sys, os, sqlite3, numpy as np, pandas as pd, joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, backend as K
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# GitHub'dan klonla
if not os.path.exists('jetxpredictor'):
    !git clone https://github.com/onndd/jetxpredictor.git
%cd jetxpredictor
sys.path.append('/content/jetxpredictor')

from category_definitions import CategoryDefinitions, FeatureEngineering
print(f"✅ Proje yüklendi - Kritik eşik: {CategoryDefinitions.CRITICAL_THRESHOLD}x")

In [ ]:
# Veritabanından veri yükle
conn = sqlite3.connect('jetx_data.db')
data = pd.read_sql_query("SELECT value FROM jetx_results ORDER BY id", conn)
conn.close()

all_values = data['value'].values
print(f"✅ {len(all_values)} veri yüklendi")
print(f"Aralık: {all_values.min():.2f}x - {all_values.max():.2f}x")

below = (all_values < 1.5).sum()
above = (all_values >= 1.5).sum()
print(f"\n🔴 CLASS IMBALANCE:")
print(f"1.5 altı: {below} ({below/len(all_values)*100:.1f}%)")
print(f"1.5 üstü: {above} ({above/len(all_values)*100:.1f}%)")
print(f"Dengesizlik: 1:{above/below:.2f}")

In [ ]:
# Feature Engineering
window_size = 500
X_f, X_50, X_200, X_500 = [], [], [], []
y_reg, y_cls, y_thr = [], [], []

for i in tqdm(range(window_size, len(all_values)-1), desc='Features'):
    hist = all_values[:i].tolist()
    target = all_values[i]
    
    feats = FeatureEngineering.extract_all_features(hist)
    X_f.append(list(feats.values()))
    X_50.append(all_values[i-50:i])
    X_200.append(all_values[i-200:i])
    X_500.append(all_values[i-500:i])
    
    y_reg.append(target)
    cat = CategoryDefinitions.get_category_numeric(target)
    onehot = np.zeros(3)
    onehot[cat] = 1
    y_cls.append(onehot)
    y_thr.append(1.0 if target >= 1.5 else 0.0)

X_f = np.array(X_f)
X_50 = np.array(X_50).reshape(-1, 50, 1)
X_200 = np.array(X_200).reshape(-1, 200, 1)
X_500 = np.array(X_500).reshape(-1, 500, 1)
y_reg = np.array(y_reg)
y_cls = np.array(y_cls)
y_thr = np.array(y_thr)

print(f"✅ {len(X_f)} örnek hazırlandı")

In [ ]:
# Normalizasyon
scaler = StandardScaler()
X_f = scaler.fit_transform(X_f)
X_50 = np.log10(X_50 + 1e-8)
X_200 = np.log10(X_200 + 1e-8)
X_500 = np.log10(X_500 + 1e-8)

# Split
idx = np.arange(len(X_f))
tr_idx, te_idx = train_test_split(idx, test_size=0.2, shuffle=False)

X_f_tr, X_50_tr, X_200_tr, X_500_tr = X_f[tr_idx], X_50[tr_idx], X_200[tr_idx], X_500[tr_idx]
y_reg_tr, y_cls_tr, y_thr_tr = y_reg[tr_idx], y_cls[tr_idx], y_thr[tr_idx]

X_f_te, X_50_te, X_200_te, X_500_te = X_f[te_idx], X_50[te_idx], X_200[te_idx], X_500[te_idx]
y_reg_te, y_cls_te, y_thr_te = y_reg[te_idx], y_cls[te_idx], y_thr[te_idx]

print(f"Train: {len(X_f_tr)}, Test: {len(X_f_te)}")

In [ ]:
# GÜÇLENDIRILMIŞ MODEL
n_f = X_f_tr.shape[1]

inp_f = layers.Input((n_f,), name='features')
inp_50 = layers.Input((50, 1), name='seq50')
inp_200 = layers.Input((200, 1), name='seq200')
inp_500 = layers.Input((500, 1), name='seq500')

# N-BEATS (güçlendirilmiş)
def nbeats(x, units, blocks, name):
    for i in range(blocks):
        x = layers.Dense(units, activation='relu')(x)
        x = layers.Dropout(0.15)(x)
    return x

nb_s = layers.Flatten()(inp_50)
nb_s = nbeats(nb_s, 64, 5, 's')
nb_s = layers.Dense(64, activation='relu')(nb_s)

nb_m = layers.Flatten()(inp_200)
nb_m = nbeats(nb_m, 128, 7, 'm')
nb_m = layers.Dense(128, activation='relu')(nb_m)

nb_l = layers.Flatten()(inp_500)
nb_l = nbeats(nb_l, 256, 9, 'l')
nb_l = layers.Dense(256, activation='relu')(nb_l)

nb_all = layers.Concatenate()([nb_s, nb_m, nb_l])

# TCN (güçlendirilmiş)
def tcn_blk(x, f, d, n):
    c = layers.Conv1D(f, 3, dilation_rate=d, padding='causal', activation='relu')(x)
    c = layers.Dropout(0.15)(c)
    r = layers.Conv1D(f, 1, padding='same')(x) if x.shape[-1] != f else x
    return layers.Add()([c, r])

tcn = inp_500
tcn = tcn_blk(tcn, 64, 1, '1')
tcn = tcn_blk(tcn, 64, 2, '2')
tcn = tcn_blk(tcn, 128, 4, '3')
tcn = tcn_blk(tcn, 128, 8, '4')
tcn = tcn_blk(tcn, 256, 16, '5')
tcn = tcn_blk(tcn, 256, 32, '6')
tcn = tcn_blk(tcn, 256, 64, '7')
tcn = tcn_blk(tcn, 512, 128, '8')
tcn = layers.GlobalAveragePooling1D()(tcn)
tcn = layers.Dense(512, activation='relu')(tcn)

# Fusion
fus = layers.Concatenate()([inp_f, nb_all, tcn])
fus = layers.Dense(512, activation='relu')(fus)
fus = layers.Dropout(0.25)(fus)
fus = layers.Dense(256, activation='relu')(fus)
fus = layers.Dropout(0.25)(fus)
fus = layers.Dense(128, activation='relu')(fus)

# Outputs
out_reg = layers.Dense(1, activation='linear', name='regression')(layers.Dense(64, activation='relu')(fus))
out_cls = layers.Dense(3, activation='softmax', name='classification')(layers.Dense(64, activation='relu')(fus))
out_thr = layers.Dense(1, activation='sigmoid', name='threshold')(layers.Dense(32, activation='relu')(fus))

model = models.Model([inp_f, inp_50, inp_200, inp_500], [out_reg, out_cls, out_thr])
print(f"✅ Model oluşturuldu: {model.count_params():,} parametre")

In [ ]:
# FOCAL LOSS
def focal_loss(gamma=2.0, alpha=0.75):
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        return -K.mean(alpha * K.pow(1 - pt, gamma) * K.log(pt))
    return loss

# ULTRA AGGRESSIVE LOSS
def ultra_loss(y_true, y_pred):
    mae = K.abs(y_true - y_pred)
    crit = K.cast(K.logical_and(y_true >= 1.4, y_true <= 1.6), 'float32') * 20.0
    wrong = K.cast(K.not_equal(K.cast(y_true >= 1.5, 'float32'), K.cast(y_pred >= 1.5, 'float32')), 'float32') * 10.0
    w = K.maximum(K.maximum(crit, wrong), 1.0)
    return K.mean(mae * w)

# Class weights
c0 = (y_thr_tr == 0).sum()
c1 = (y_thr_tr == 1).sum()
w0 = (len(y_thr_tr) / (2 * c0)) * 2.5
w1 = len(y_thr_tr) / (2 * c1)
print(f"\nClass weights: 1.5 altı={w0:.2f}, 1.5 üstü={w1:.2f}")

model.compile(
    optimizer=Adam(0.0005),
    loss={'regression': ultra_loss, 'classification': 'categorical_crossentropy', 'threshold': focal_loss()},
    loss_weights={'regression': 0.3, 'classification': 0.2, 'threshold': 0.5},
    metrics={'regression': ['mae'], 'classification': ['accuracy'], 'threshold': ['accuracy']}
)
print("✅ Model compiled")

In [ ]:
# CALLBACKS
class MetricsCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
            p = self.model.predict([X_f_tr[:500], X_50_tr[:500], X_200_tr[:500], X_500_tr[:500]], verbose=0)[2].flatten()
            pb = (p >= 0.5).astype(int)
            tb = y_thr_tr[:500].astype(int)
            below = tb == 0
            above = tb == 1
            ba = (pb[below] == tb[below]).mean() if below.sum() > 0 else 0
            aa = (pb[above] == tb[above]).mean() if above.sum() > 0 else 0
            print(f"\nEpoch {epoch+1}: 1.5 ALTI={ba*100:.1f}%, ÜSTÜ={aa*100:.1f}%")

cb = [
    callbacks.ModelCheckpoint('jetx_best.h5', monitor='val_threshold_accuracy', save_best_only=True, mode='max', verbose=1),
    callbacks.EarlyStopping(monitor='val_threshold_accuracy', patience=40, mode='max', restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7),
    MetricsCallback()
]
print("✅ Callbacks hazır")

In [ ]:
# TRAINING
print("🚀 Eğitim başlıyor: 300 epoch, batch=16, patience=40\n")

hist = model.fit(
    [X_f_tr, X_50_tr, X_200_tr, X_500_tr],
    {'regression': y_reg_tr, 'classification': y_cls_tr, 'threshold': y_thr_tr},
    epochs=300,
    batch_size=16,
    validation_split=0.2,
    callbacks=cb,
    verbose=1
)

print("\n✅ Eğitim tamamlandı!")

In [ ]:
# EVALUATION
pred = model.predict([X_f_te, X_50_te, X_200_te, X_500_te], verbose=0)
p_reg = pred[0].flatten()
p_cls = pred[1]
p_thr = pred[2].flatten()

mae = mean_absolute_error(y_reg_te, p_reg)
rmse = np.sqrt(mean_squared_error(y_reg_te, p_reg))

thr_true = (y_reg_te >= 1.5).astype(int)
thr_pred = (p_thr >= 0.5).astype(int)
thr_acc = accuracy_score(thr_true, thr_pred)

below_mask = thr_true == 0
above_mask = thr_true == 1
below_acc = accuracy_score(thr_true[below_mask], thr_pred[below_mask])
above_acc = accuracy_score(thr_true[above_mask], thr_pred[above_mask])

cls_true = np.argmax(y_cls_te, axis=1)
cls_pred = np.argmax(p_cls, axis=1)
cls_acc = accuracy_score(cls_true, cls_pred)

cm = confusion_matrix(thr_true, thr_pred)

print("\n" + "="*60)
print("TEST SETİ SONUÇLARI")
print("="*60)
print(f"\nMAE: {mae:.4f} | RMSE: {rmse:.4f}")
print(f"\n🔴 1.5 ALTI Doğruluk: {below_acc*100:.2f}% (Hedef: 70%+)")
if below_acc >= 0.70:
    print("✅ ✅ ✅ HEDEF BAŞARILI! Para kaybı riski minimize edildi!")
else:
    print("⚠️ Hedefin altında - daha fazla eğitim gerekebilir")
print(f"\n🟢 1.5 ÜSTÜ Doğruluk: {above_acc*100:.2f}%")
print(f"📊 Genel Threshold Acc: {thr_acc*100:.2f}%")
print(f"📁 Kategori Acc: {cls_acc*100:.2f}%")

print(f"\n📋 CONFUSION MATRIX:")
print(f"Doğru 1.5 altı: {cm[0,0]} | Yanlış 1.5 üstü: {cm[0,1]} ⚠️")
print(f"Yanlış 1.5 altı: {cm[1,0]} | Doğru 1.5 üstü: {cm[1,1]}")

if cm[0,0] + cm[0,1] > 0:
    fpr = cm[0,1] / (cm[0,0] + cm[0,1])
    print(f"\n⚠️ PARA KAYBI RİSKİ: {fpr*100:.1f}% (1.5 altıyken yanlış üstü tahmini)")
    if fpr < 0.30:
        print("✅ İyi! Para kaybı riski düşük")
    else:
        print("❌ Yüksek risk!")

In [ ]:
# KAYDET & İNDİR
model.save('jetx_model.h5')
joblib.dump(scaler, 'scaler.pkl')

import json
info = {
    'model': 'AGGRESSIVE_NBEATS_TCN',
    'params': int(model.count_params()),
    'epochs': len(hist.history['loss']),
    'threshold_acc': float(thr_acc),
    'below_15_acc': float(below_acc),
    'above_15_acc': float(above_acc),
    'class_acc': float(cls_acc),
    'mae': float(mae)
}

with open('model_info.json', 'w') as f:
    json.dump(info, f, indent=2)

print("✅ Dosyalar kaydedildi")
print(json.dumps(info, indent=2))

try:
    from google.colab import files
    files.download('jetx_model.h5')
    files.download('scaler.pkl')
    files.download('model_info.json')
    print("\n✅ Dosyalar indirildi!")
except:
    print("\n⚠️ Colab dışında - dosyalar sadece kaydedildi")

if below_acc >= 0.70:
    print("\n🎉 Model hazır! 1.5 altı tahminlerinde hedefi aştı!")
else:
    print("\n⚠️ Model daha fazla eğitim gerektirebilir")